In [57]:
import pandas as pd
import re
import numpy as np
import heapq
import nltk
from nltk.corpus import stopwords
from naive_bayes import naive_bayes_train, naive_bayes_predict

# First Time Running

1. install nltk and download stopwords

In [ ]:
%pip install nltk

In [ ]:
nltk.download('stopwords')

In [56]:
!pwd
raw_train_df=pd.read_csv("./data/train.csv") 
raw_test_df=pd.read_csv("./data/test.csv") 

raw_train_df.head()


/home/marxw/Desktop/FakeNews


,title,text,subject,date,label
0,Clinton faces pressure to pick VP who is tough...,WASHINGTON (Reuters) - Members of the Democrat...,politicsNews,"July 21, 2016",real
1,"Ryan, Trump cite 'positive step' toward Republ...",WASHINGTON (Reuters) - Presumptive Republican ...,politicsNews,"May 12, 2016",real
2,WATCH: President Obama Dares Republicans To S...,Conservatives talk the talk but can they walk ...,News,"July 9, 2016",fake
3,Hariri warns Lebanon faces Arab sanctions risk...,BEIRUT (Reuters) - Saad al-Hariri warned on Su...,worldnews,"November 12, 2017",real
4,A POEM: ‘Twas The Night Before CNN’s Christmas…’,ACR s BOILER ROOM presents a Christmas poem Tw...,Middle-east,"December 25, 2017",fake


In [ ]:
raw_train_df['text']

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
    return text

In [ ]:
x_train = []
for t in list(raw_train_df['text']):
    x_train.append(clean_text(t))
    
x_test= []
for t in list(raw_test_df['text']):
    x_test.append(clean_text(t))

In [ ]:
#for unit testing
x_unit = []
x_unit.append(clean_text(x_train[1]))
x_unit.append(clean_text(x_train[2]))
x_unit.append(clean_text(x_train[3]))
x_unit

In [ ]:
def extract_text(s):    
    stop_words = set(stopwords.words('english'))  #pending more stop words    
    words = re.sub("[^\w]", " ",  s).split()    
    extracted = [w.lower() for w in words if w not in stop_words]    
    return extracted

In [ ]:
def init_bag_dict(x):
    bag = {}
    x = list(set(x))
    for entry in x:
        new_words = extract_text(entry)
        for word in new_words:
            if word not in bag.keys():
                bag[word] = 1
            else:
                bag[word] += 1
    return bag

In [45]:
def tokenize(x, bag, d=300):
    most_freq = heapq.nlargest(d, bag, key=bag.get)
    vectorized_x = []
    for entry in x:
        curr_bag = np.zeros(len(most_freq))
        extracted_text = extract_text(entry)
        for i, token in enumerate(most_freq):
            curr_bag[i] = extracted_text.count(token)
        vectorized_x.append(curr_bag)
    return vectorized_x, most_freq

In [ ]:
# def init_BOW(x):
#     bag = []
#     x = list(set(x))
#     for entry in x:
#         new_words = extract_text(entry)
#         bag.extend(new_words)
#     return bag

In [ ]:
# def BOW_tokenize(x):
#     vectorized_x = []
#     bag = init_BOW(x)
#     for entry in x:
#         extracted_text = extract_text(entry)
#         curr_bag = np.zeros(len(bag))
#         for w in extracted_text:
#             for i, word_in_bag in enumerate(bag): 
#                 if(w==word_in_bag):
#                     curr_bag[i] += 1
#         vectorized_x.append(curr_bag)
#     return vectorized_x

In [46]:
bag = init_bag_dict(x_train)

In [51]:
vectorized_x, most_freq = tokenize(x_train, bag)

In [52]:
vectorized_x

[array([13.,  4.,  5.,  6.,  0.,  3.,  0.,  1.,  1.,  1.,  1.,  4.,  0.,
         1., 18.,  0.,  1.,  4.,  1.,  0.,  0.,  0.,  1.,  0.,  2.,  0.,
         3.,  1.,  4.,  0.,  1.,  0.,  2.,  1.,  1.,  2.,  2.,  2.,  0.,
         1.,  3.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
         0.,  1.,  0.,  0.,  1.,  4.,  2.,  0.,  1.,  1.,  2.,  0.,  0.,
         2.,  0.,  1.,  0.,  8.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  3.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,
         3.,  1.,  3.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  0.,  5.,  4.,  2.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.

In [53]:
y_train = raw_train_df['label']
y_train = np.array(list(map(lambda x: 1 if x=="real" else 0, y_train)))


In [54]:
y_train

array([1, 1, 0, ..., 1, 0, 1])

In [60]:
vectorized_x = np.asarray(vectorized_x)
vectorized_x.shape

(39898, 300)

In [ ]:

nb_params = {}
nb_model = naive_bayes_train(vectorized_x.T, y_train, nb_params)